In [3]:
import os
import cv2
import numpy as np
from cvl.dataset import OnlineTrackingBenchmark
from cvl.trackers import MOSSETracker, DCFMOSSETracker
from cvl.lib import get_roi, resume_performance, get_arguments
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import torch
from tqdm.notebook import tqdm as tqdm
import json

In [2]:
ds_path = "Mini-OTB/"

In [3]:
seq_list = os.listdir(ds_path)

In [4]:
dataset = OnlineTrackingBenchmark(ds_path)

# 1) Choose the tracker

In [5]:
TRACKER = "vgg16" # ["mosse", "resnet", "mobilenet", "alexnet", "vgg16", "hog"]

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
if TRACKER == "mosse":
    tracker = MOSSETracker()
    squared = False
    bigger = True
elif TRACKER in ["resnet", "mobilenet", "alexnet", "vgg16", "hog"]:
    tracker = DCFMOSSETracker(dev = device, features = TRACKER)
    squared = True
    bigger = True

In [8]:
results = {}

In [ ]:
for idx, seq in enumerate(dataset):
    print(f"----------------------------Evaluating video named {seq.sequence_name}----------------------------------")
    bboxes = []
    for frame_idx, frame in enumerate(seq):
        image_color = frame['image']
        if TRACKER == "mosse":
            image = np.sum(image_color, 2) / 3 # grayscale
        else:
            image = np.transpose(np.float64(image_color), (2, 0, 1))
        if frame_idx == 0:
            bbox = copy(frame['bounding_box'])
            aaaa = bbox
            if bbox.width % 2 == 0:
                bbox.width += 1
            if bbox.height % 2 == 0:
                bbox.height += 1
            roi = get_roi(bbox, squared = squared, bigger = bigger) # get roi slightly bigger that bbox
            tracker.start(image, bbox, roi) # first frame approach
        else:
            tracker.detect(image)
            tracker.update()
        bboxes.append(copy(tracker.bbox))
    ious = dataset.calculate_per_frame_iou(idx, bboxes)
    print("mIoU:", np.mean(ious))
    results[idx] = np.mean(ious)

----------------------------Evaluating video named Basketball----------------------------------
mIoU: 0.8701828133552272
----------------------------Evaluating video named Biker----------------------------------
mIoU: 0.24090797652104243
----------------------------Evaluating video named BlurBody----------------------------------
mIoU: 0.11383801583045508
----------------------------Evaluating video named BlurCar3----------------------------------
mIoU: 0.521603729533113
----------------------------Evaluating video named Bolt----------------------------------
mIoU: 0.013513747939977448
----------------------------Evaluating video named Box----------------------------------
mIoU: 0.42211570608579363
----------------------------Evaluating video named CarScale----------------------------------
mIoU: 0.3271556791449739
----------------------------Evaluating video named Coke----------------------------------
mIoU: 0.6678387945863345
----------------------------Evaluating video named Coupon-

In [13]:
result_file = f"../results/ious/experiments/2/best_variation_layers/vgg16/block1.json"

In [10]:
with open(result_file, 'w') as fp:
    json.dump(results, fp)

NameError: name 'results' is not defined

# 2) Load results

In [15]:
with open(result_file) as fp:
    data = json.load(fp)

In [16]:
mIoU = 0

In [17]:
for k,v in data.items():
    mIoU += v

In [18]:
k 

'29'

In [19]:
mIoU / 30

0.3524316212008757